In [15]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)

In [14]:
#reading the data
temp <- tempfile()
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00368/Facebook_metrics.zip",temp)

#Reading the data in R
facebook <- read_csv2(unz(temp, "dataset_Facebook.csv"))
head(facebook)
unlink(temp)

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.

Parsed with column specification:
cols(
  `Page total likes` = col_double(),
  Type = col_character(),
  Category = col_double(),
  `Post Month` = col_double(),
  `Post Weekday` = col_double(),
  `Post Hour` = col_double(),
  Paid = col_double(),
  `Lifetime Post Total Reach` = col_double(),
  `Lifetime Post Total Impressions` = col_double(),
  `Lifetime Engaged Users` = col_double(),
  `Lifetime Post Consumers` = col_double(),
  `Lifetime Post Consumptions` = col_double(),
  `Lifetime Post Impressions by people who have liked your Page` = col_double(),
  `Lifetime Post reach by people who like your Page` = col_double(),
  `Lifetime People who have liked your Page and engaged with your post` = col_double(),
  comment = col_double(),
  like = col_double(),
  share = col_double(),
  `Total Interactions` = col_double()
)



Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
139441,Photo,2,12,4,3,0,2752,5091,178,109,159,3078,1640,119,4,79,17,100
139441,Status,2,12,3,10,0,10460,19057,1457,1361,1674,11710,6112,1108,5,130,29,164
139441,Photo,3,12,3,3,0,2413,4373,177,113,154,2812,1503,132,0,66,14,80
139441,Photo,2,12,2,10,1,50128,87991,2211,790,1119,61027,32048,1386,58,1572,147,1777
139441,Photo,2,12,2,3,0,7244,13594,671,410,580,6228,3200,396,19,325,49,393
139441,Status,2,12,1,9,0,10472,20849,1191,1073,1389,16034,7852,1016,1,152,33,186


In [4]:
#Splitting into training and testing data
#Split into training (75 percent of data set) and test data
facebook_split <- initial_split(facebook, prop = 0.75, strata = `Total Interactions`)
facebook_train <- training(facebook_split)
facebook_test <- testing(facebook_split)

In [16]:
#Finding best K in Hour of Day through training data
#Recipe
facebook_recipe <- recipe(`Post Month` ~ comment + share + like, data = facebook_train) %>%
  step_scale(all_predictors()) %>%
  step_center(all_predictors())

#Model
spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
  set_engine("kknn") %>%
  set_mode("regression")

#Performing a 5 fold cross validation 
facebook_vfold <- vfold_cv(facebook_train, v = 5, strata = `Post Month`)

#Workflow
fb_workflow <- workflow() %>%
              add_recipe(facebook_recipe) %>%
              add_model(spec)
fb_workflow
#Testing 200 values of K in a tibble
gridvals <- tibble(neighbors = seq(1:200))

#(Tunes workflow to test all values of K, resamples cross-validation data set, collects the statistics from that model)
fb_results <- fb_workflow %>%
  tune_grid(resamples = facebook_vfold, grid = gridvals) %>%
  collect_metrics()
fb_results

#Finding min amount of RMSPE
facebook_min <- fb_results %>%
                filter(.metric == "rmse") %>%
                filter(mean == min(mean))
facebook_min

══ Workflow ════════════════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: nearest_neighbor()

── Preprocessor ────────────────────────────────────────────────────────────────
2 Recipe Steps

● step_scale()
● step_center()

── Model ───────────────────────────────────────────────────────────────────────
K-Nearest Neighbor Model Specification (regression)

Main Arguments:
  neighbors = tune()
  weight_func = rectangular

Computational engine: kknn 


x Fold1: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold5: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...



neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,rmse,standard,4.2127464,3,0.18027845,Model001
1,rsq,standard,0.0264256,3,0.01839504,Model001
2,rmse,standard,3.7642177,3,0.13630995,Model002
⋮,⋮,⋮,⋮,⋮,⋮,⋮
199,rsq,standard,0.0357319,3,0.02365874,Model199
200,rmse,standard,3.2045850,3,0.05082966,Model200
200,rsq,standard,0.0347593,3,0.01850892,Model200


neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
23,rmse,standard,3.131536,3,0.08291609,Model023


In [17]:
k_min <- facebook_min %>%
         pull(neighbors)

facebook_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) %>%
         set_engine("kknn") %>%
         set_mode("regression")

fb_best_fit <- workflow() %>%
         add_recipe(facebook_recipe) %>%
         add_model(facebook_best_spec) %>%
         fit(data = facebook_train)

fb_summary <- fb_best_fit  %>%
              predict(facebook_test) %>%
              bind_cols(facebook_test) %>%
             metrics(truth = `Post Hour`, estimate = .pred)
fb_summary

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,4.03681553
rsq,standard,0.02806872
mae,standard,3.67267586


In [25]:
#Splitting into training and testing data
#Split into training (75 percent of data set) and test data
facebook_split_2 <- initial_split(facebook, prop = 0.75, strata = `Post Hour`)
facebook_train_2 <- training(facebook_split_2)
facebook_test_2 <- testing(facebook_split_2)

#Recipe for Post Hour
facebook_recipe_2 <- recipe(`Post Hour` ~ comment + share + like, data = facebook_train_2) %>%
                   step_scale(all_predictors()) %>%
                   step_center(all_predictors())
#Model
facebook_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
                 set_engine("kknn") %>%
                 set_mode("regression")

#5 fold cross validation
facebook_vfold_2 <- vfold_cv(facebook_train_2, v = 5, strata = `Post Hour`)

#workflow
facebook_workflow_2 <- workflow() %>%
                     add_recipe(facebook_recipe_2) %>%
                     add_model(facebook_spec_2)

#testing 200 values to find the best value for K
gridvals_2 <- tibble(neighbors = seq(1, 200))

#Tuning workflow to test the values of K
facebook_results_2 <- facebook_workflow_2 %>%
                        tune_grid(resamples = facebook_vfold_2, grid = gridvals_2) %>%
                        collect_metrics()
#Min amount of RMSPE along with mean and standard error to find the best K value to use
facebook_min_2 <- facebook_results_2 %>%
    filter(.metric == "rmse") %>%
    arrange(mean)  %>% 
    slice(1)

facebook_min_2

#Min RMSPE
k_min_2 <- facebook_min_2 %>%
            pull(neighbors)

#Using the best K in our model
facebook_best_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min_2) %>%
                            set_engine("kknn") %>%
                            set_mode("regression")

#Reworking the workflow to incorporate our chosen K
facebook_best_fit_2 <- workflow() %>%
                        add_recipe(facebook_recipe_2) %>%
                        add_model(facebook_best_spec_2) %>%
                        fit(data = facebook_train_2)

#Summary of the training vs testing data
facebook_summary_2 <- facebook_best_fit_2 %>% 
                       predict(facebook_test_2) %>%
                       bind_cols(facebook_test_2) %>%
                       metrics(truth = `Post Hour`, estimate = .pred) 

facebook_summary_2

x Fold3: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold4: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...

x Fold5: model 1/1 (predictions): Error: Problem with `mutate()` input `.row`.
[...



neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
137,rmse,standard,4.25447,2,0.03878898,Model137


ERROR: Error: Can't recycle `..1` (size 122) to match `..2` (size 123).
